In [1]:
import geopandas as gpd
import openeo 
import json

In [2]:
connection = openeo.connect(url="openeo.dataspace.copernicus.eu").authenticate_oidc()

Authenticated using refresh token.


In [3]:
aoi = json.loads(gpd.read_file("Dataset/Extent/StudyArea.gpkg").to_json())

In [4]:
sentinel2_cube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent=aoi,
    temporal_extent=["2019-10-13", "2019-11-05"],
    bands=["B02","B03", "B04", "B08"],
)

In [5]:
sentinel1_pre_cube = connection.load_collection(
    "SENTINEL1_GRD",
    spatial_extent=aoi,
    temporal_extent=["2019-09-01", "2019-10-01"],
    bands = ['VV'],
    properties={"sat:orbit_state": lambda v: v=="ascending"}
)
sentinel1_pre_cube = sentinel1_pre_cube.sar_backscatter(coefficient="gamma0-ellipsoid")

In [6]:
sentinel1_post_cube = connection.load_collection(
    "SENTINEL1_GRD",
    spatial_extent=aoi,
    temporal_extent=["2019-10-13", "2019-11-05"],
    bands = ['VV'],
    properties={"sat:orbit_state": lambda v: v=="ascending"}
)
sentinel1_post_cube = sentinel1_post_cube.sar_backscatter(coefficient="gamma0-ellipsoid")

In [7]:
sentinel2_cube  = sentinel2_cube.reduce_dimension(dimension="t", reducer="median")

In [8]:
sentinel1_pre_cube = sentinel1_pre_cube.reduce_dimension(dimension="t", reducer="last")

In [9]:
sentinel1_post_cube = sentinel1_post_cube.reduce_dimension(dimension="t", reducer="first")

In [10]:
ratio = sentinel1_post_cube.divide(sentinel1_pre_cube)

#rename for merging
ratio = ratio.rename_labels(dimension="bands", target=["ratio"])
sentinel1_post_cube = sentinel1_post_cube.rename_labels(dimension="bands", target=["postSAR"])
sentinel1_pre_cube = sentinel1_pre_cube.rename_labels(dimension="bands", target=["preSAR"])

In [11]:
radar_cubes = [sentinel1_pre_cube,sentinel1_post_cube,ratio]
merged_cube = sentinel2_cube
for cube in radar_cubes:
    merged_cube = merged_cube.merge_cubes(cube)

In [12]:
job_options = {
    "executor-memory": "3G",
    "executor-memoryOverhead": "4G",
    "executor-cores": "2",
}
merged_cube.execute_batch(title="Download Japan",outputfile="Data/JapanImages.tiff", job_options=job_options)

0:00:00 Job 'j-2407096d8d4c4200919bd10d5f1c6a79': send 'start'
0:00:15 Job 'j-2407096d8d4c4200919bd10d5f1c6a79': queued (progress 0%)
0:00:20 Job 'j-2407096d8d4c4200919bd10d5f1c6a79': queued (progress 0%)
0:00:26 Job 'j-2407096d8d4c4200919bd10d5f1c6a79': queued (progress 0%)
0:00:34 Job 'j-2407096d8d4c4200919bd10d5f1c6a79': queued (progress 0%)
0:00:44 Job 'j-2407096d8d4c4200919bd10d5f1c6a79': queued (progress 0%)
0:00:57 Job 'j-2407096d8d4c4200919bd10d5f1c6a79': queued (progress 0%)
0:01:17 Job 'j-2407096d8d4c4200919bd10d5f1c6a79': running (progress N/A)
0:01:37 Job 'j-2407096d8d4c4200919bd10d5f1c6a79': running (progress N/A)
0:02:01 Job 'j-2407096d8d4c4200919bd10d5f1c6a79': running (progress N/A)
0:02:31 Job 'j-2407096d8d4c4200919bd10d5f1c6a79': running (progress N/A)
0:03:08 Job 'j-2407096d8d4c4200919bd10d5f1c6a79': running (progress N/A)
0:03:55 Job 'j-2407096d8d4c4200919bd10d5f1c6a79': running (progress N/A)
0:04:54 Job 'j-2407096d8d4c4200919bd10d5f1c6a79': finished (progress 100%

<BatchJob job_id='j-2407096d8d4c4200919bd10d5f1c6a79'>